In [1]:
import pandas as pd
import re

In [2]:
Data = pd.read_csv("JobDataTranslated.csv")
keyWords = pd.read_csv("words_set_long.csv")
keyWords_short = pd.read_csv("words_set_short.csv")
Data=Data[:-20]

In [3]:
# just a thing to clean input
#Data['Job Description'].apply(lambda x: x.lower())

def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    
    return text

# preprocessing speeches
Data['Cleaned Job Description'] = Data['Job Description'].apply(clean)

from string import punctuation

def remove_punctuation(s):
    global punctuation
    for p in punctuation:
        s = s.replace(p, '')
    return s

#Data['Cleaned Job Description'] = Data['Cleaned Job Description'].map(remove_punctuation)

Data['Cleaned Job Description'] = Data['Cleaned Job Description'].apply(lambda x: x.lower())

In [4]:
kWords=[keyWords_short['0'][i] for i in range(len(keyWords_short))]
#kWords=[keyWords['0'][i] for i in range(len(keyWords))]

In [5]:
#Data['Cleaned Job Description']

In [40]:

""" this thing allows us to find all words start indexes"""
def find_all(text, sub):
    start = 0
    while True:
        start = text.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

"""all words start indexes for list of words"""
def find_all_in_list(text, subs: list):
    l=[]
    for i in subs:
        l.append(list(find_all(text,i)))
    m = [l[j][k] for j in range(len(l)) for k in range(len(l[j]))]
    return m


"gives a sentence for an index of word inside"
def get_sentence(num, text ):
    l= find_all_in_list(text, ['.','!','?',';'])
    if (len(l)==0):
        return -1
    
    l.sort()
    if (num <l[0]):
        return text[0:l[0]]
    elif (num>l[-1]):
        return text[l[-1]+1:]
    
    x=[i for i in range(len(l)) if l[i]<num and l[i+1]>num]
    return (text[l[x[0]]+1:l[x[0]+1]])


In [41]:
l = []
for d in Data['Cleaned Job Description'] :
    for i in kWords:
        if d.find(i)!=-1:
            #print(d.find(i),d.find(i)+len(i), get_sentence(d.find(i),d) ,'hui',i)
            s = (get_sentence(d.find(i),d))
            if (s!=-1):
                l.append((s ,{"entities": [(s.find(i),s.find(i)+len(i), "SKILL")]}))

In [42]:
l[30:40]

[(' you are partly responsible for the implementation and you will mainly perform system management activities',
  {'entities': [(86, 107, 'SKILL')]}),
 (' experience in installation technology', {'entities': [(15, 38, 'SKILL')]}),
 (' and do you want to be trained as an experienced service technician',
  {'entities': [(49, 67, 'SKILL')]}),
 (' as a starter you will be trained within the organization as a service technician in heating technology',
  {'entities': [(85, 103, 'SKILL')]}),
 (' and do you want to be trained as an experienced service technician',
  {'entities': [(37, 67, 'SKILL')]}),
 (' as an apprentice service technician you work in a pleasant, dynamic team with a pleasant working atmosphere',
  {'entities': [(7, 36, 'SKILL')]}),
 (' it is an advantage if you have completed an mbo level 2 education in electricity',
  {'entities': [(45, 54, 'SKILL')]}),
 (' as a technical support employee, we give you the right intensive training and coaching',
  {'entities': [(6, 23, 'SKIL

In [44]:
len(l)

7183

In [45]:
import json
with open("training_set_1.txt", "w") as fp:
    json.dump(l, fp)

    
# that's how you load this thing, the format will be mimimally diffreent so either fix it or just merge 2 notebooks
#with open("training_set_1.txt", "r") as fp:
    #b = json.load(fp)